In [1]:
import sys
!{sys.executable} -m pip install --upgrade keras

    100% |████████████████████████████████| 327kB 3.3MB/s eta 0:00:01
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python3.6/dist-packages (from keras)
  Using cached six-1.11.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 12.2MB 127kB/s eta 0:00:01
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: PyYAML 3.11
    Uninstalling PyYAML-3.11:
      Successfully uninstalled PyYAML-3.11
  Found existing installation: numpy 1.14.0
    Uninstalling numpy-1.14.0:
      Successfully uninstalled numpy-1.14.0
  Found existing installation: Keras 2.0.6
    Uninstalling Keras-2.0.6:
      Successfully uninstalled Keras-2.0.6


In [2]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers import concatenate, CuDNNGRU
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
        
# path = '..'
# EMBEDDING_FILE=path+'/input/glove.840B.300d.txt'
# EMBEDDING_FILE=path+'/input/crawl-300d-2M.vec'
# TRAIN_DATA_FILE=path+'/input/train.csv'
# TEST_DATA_FILE=path+'/input/test.csv'

EMBEDDING_FILE='/public/models/glove/glove.840B.300d.txt'
TRAIN_DATA_FILE='/public/toxic_comments/train.csv'
TEST_DATA_FILE='/public/toxic_comments/test.csv'

MAX_SEQUENCE_LENGTH = 150
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300
num_dense = 256
rate_drop_lstm = 0.2
rate_drop_dense = 0.2

act = 'relu'

In [4]:
def cleanData(text, stemming = False, lemmatize=False):    
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])
    return text

In [5]:
print('Indexing word vectors')

count = 0
embeddings_index = {}
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs.reshape(-1)
    coef = embeddings_index[word]
f.close()

print('Found %d word vectors of glove.' % len(embeddings_index))
emb_mean,emb_std = coef.mean(), coef.std()
print(emb_mean,emb_std)

print('Total %s word vectors.' % len(embeddings_index))

train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

print('Processing text dataset')

train_df['comment_text'] = train_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                            lemmatize=False))
test_df['comment_text'] = test_df['comment_text'].map(lambda x: cleanData(x,  stemming = False, 
                                                                          lemmatize=False))

#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)
#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    return(text)


list_sentences_train = train_df["comment_text"].fillna("NA").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
list_sentences_test = test_df["comment_text"].fillna("NA").values


comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))
    
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Indexing word vectors
Found 2195895 word vectors of glove.
-0.01444638 0.47249147
Total 2195895 word vectors.
Processing text dataset
Found 292462 unique tokens
Shape of data tensor: (159571, 150)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 150)


In [6]:
data_post = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post', truncating='post')
print('Shape of data tensor:', data_post.shape)
print('Shape of label tensor:', y.shape)

test_data_post = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of test_data tensor:', test_data_post.shape)

Shape of data tensor: (159571, 150)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 150)


In [7]:
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 21603


In [8]:
import sys
from os.path import dirname
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K


class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [9]:
from keras.optimizers import Nadam
def deepmoji_architecture(nb_classes, maxlen, feature_output=False, embed_dropout_rate=0, final_dropout_rate=0, embed_l2=1E-6, return_attention=False):
    """
    Returns the DeepMoji architecture uninitialized and
    without using the pretrained model weights.
    # Arguments:
        nb_classes: Number of classes in the dataset.
        nb_tokens: Number of tokens in the dataset (i.e. vocabulary size).
        maxlen: Maximum length of a token.
        feature_output: If True the model returns the penultimate
                        feature vector rather than Softmax probabilities
                        (defaults to False).
        embed_dropout_rate: Dropout rate for the embedding layer.
        final_dropout_rate: Dropout rate for the final Softmax layer.
        embed_l2: L2 regularization for the embedding layerl.
    # Returns:
        Model with the given parameters.
    """
    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding
    model_input = Input(shape=(maxlen,), dtype='int32')
    model_input_post = Input(shape=(maxlen,), dtype='int32')
    
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None
    embed = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        embeddings_regularizer=embed_reg,
        trainable=False)
    x = embed(model_input)
    x1 = Activation('tanh')(x)
    x2 = Activation('relu')(x)
    x = concatenate([x1,x2])

    # entire embedding channels are dropped out instead of the
    # normal Keras embedding dropout, which drops all channels for entire words
    # many of the datasets contain so few words that losing one or more words can alter the emotions completely
    if embed_dropout_rate != 0:
        embed_drop = SpatialDropout1D(embed_dropout_rate, name='embed_drop')
        x = embed_drop(x)

    # skip-connection from embedding to output eases gradient-flow and allows access to lower-level features
    # ordering of the way the merge is done is important for consistency with the pretrained model
    lstm_0_output = Bidirectional(CuDNNGRU(64, return_sequences=True), name="bi_gru_0")(x)
    lstm_0_output = Dropout(0.4)(lstm_0_output)
    lstm_1_output = Bidirectional(CuDNNGRU(64, return_sequences=True), name="bi_gru_1")(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])

    # if return_attention is True in AttentionWeightedAverage, an additional tensor
    # representing the weight at each timestep is returned
    weights = None
    x = AttentionWeightedAverage(name='attlayer', return_attention=return_attention)(x)
    if return_attention:
        x, weights = x

    if not feature_output:
        # output class probabilities
        if final_dropout_rate != 0:
            x = Dropout(final_dropout_rate)(x)
            
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None
    embed_post = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        embeddings_regularizer=embed_reg,
        trainable=False)
    x_post = embed(model_input_post)
    x1_post = Activation('tanh')(x_post)
    x2_post = Activation('relu')(x_post)
    x_post = concatenate([x1_post,x2_post])

    # entire embedding channels are dropped out instead of the
    # normal Keras embedding dropout, which drops all channels for entire words
    # many of the datasets contain so few words that losing one or more words can alter the emotions completely
    if embed_dropout_rate != 0:
        embed_drop_post = SpatialDropout1D(embed_dropout_rate, name='embed_drop_p')
        x_post = embed_drop_post(x_post)

    # skip-connection from embedding to output eases gradient-flow and allows access to lower-level features
    # ordering of the way the merge is done is important for consistency with the pretrained model
    lstm_0_output_post = Bidirectional(CuDNNGRU(64, return_sequences=True), name="bi_gru_0_p")(x_post)
    lstm_0_output_post = Dropout(0.4)(lstm_0_output_post)
    lstm_1_output_post = Bidirectional(CuDNNGRU(64, return_sequences=True), name="bi_gru_1_p")(lstm_0_output_post)
    x_post = concatenate([lstm_1_output_post, lstm_0_output_post, x_post])

    # if return_attention is True in AttentionWeightedAverage, an additional tensor
    # representing the weight at each timestep is returned
    weights = None
    x_post = AttentionWeightedAverage(name='attlayer_p', return_attention=return_attention)(x_post)
    if return_attention:
        x_post, weights = x_post

    if not feature_output:
        # output class probabilities
        if final_dropout_rate != 0:
            x_post = Dropout(final_dropout_rate)(x_post)
            
    x_all = concatenate([x,x_post])
    
    if not feature_output:
        # output class probabilities
        if nb_classes > 2:
            outputs = [Dense(nb_classes, activation='sigmoid', name='sigmoid_multiclass')(x_all)]
        else:
            outputs = [Dense(1, activation='sigmoid', name='sigmoid_single_class')(x_all)]
    else:
        # output penultimate feature vector
        outputs = [x]

    if return_attention:
        # add the attention weights to the outputs if required
        outputs.append(weights)
        
    model = Model(inputs=[model_input,model_input_post], outputs=outputs, name="DeepMoji")
    model.compile(loss='binary_crossentropy',
        optimizer='Adam',
        metrics=['accuracy'])

    return model

In [ ]:
# from keras.optimizers import Adam

# def get_model():
#     comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,))

#     emb = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], 
#                   input_length=MAX_SEQUENCE_LENGTH)(comment_input)
#     emb = SpatialDropout1D(0.3)(emb)
# #     x = Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(emb)
#     x = Bidirectional(CuDNNGRU(100, return_sequences=True))(emb)
#     x = Attention(MAX_SEQUENCE_LENGTH)(x)
#     x = Dense(75, activation="relu")(x)
#     x = Dropout(0.2)(x)
#     preds = Dense(6, activation="sigmoid")(x)
#     model = Model(inputs=[comment_input], outputs=preds)
#     model.compile(loss='binary_crossentropy', optimizer=Adam(clipvalue=1, clipnorm=1), metrics=['accuracy'])
#     return model


from sklearn.metrics import log_loss
import numpy as np

test_predicts_list = []

def train_folds(data,data_post, y, fold_count, batch_size):
    print("Starting to train models...")
    fold_size = len(data) // fold_count
    models = []
    for fold_id in range(6, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(data)

        print("Fold {0}".format(fold_id))
        
        train_x = np.concatenate([data[:fold_start], data[fold_end:]])
        train_xp = np.concatenate([data_post[:fold_start], data_post[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = data[fold_start:fold_end]
        val_xp = data_post[fold_start:fold_end]
        val_y = y[fold_start:fold_end]
        
        file_path="attngru_fold{0}.h5".format(fold_id)
#         model = get_model()
        model = deepmoji_architecture(6, MAX_SEQUENCE_LENGTH, feature_output=False, embed_dropout_rate=0.4, final_dropout_rate=0.4, embed_l2=0, 
                              return_attention=False)
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
        callbacks_list = [checkpoint, early] 

        hist = model.fit([train_x, train_xp], train_y, epochs=15, batch_size=128, shuffle=True, 
                         validation_data=([val_x, val_xp], val_y), callbacks = callbacks_list, verbose=1)
        model.load_weights(file_path)
        best_score = min(hist.history['val_loss'])
        
        print("Fold {0} loss {1}".format(fold_id, best_score))
        print("Predicting validation...")
        val_predicts_path = "dm_val_predicts{0}.npy".format(fold_id)
        val_predicts = model.predict([val_x, val_xp], batch_size=1024, verbose=1)
        np.save(val_predicts_path, val_predicts)
        
        print("Predicting results...")
        test_predicts_path = "dm_test_predicts{0}.npy".format(fold_id)
        test_predicts = model.predict([test_data, test_data_post], batch_size=1024, verbose=1)
        test_predicts_list.append(test_predicts)
        np.save(test_predicts_path, test_predicts)

In [ ]:
train_folds(data, data_post, y, 10, 256)

Starting to train models...
Fold 0
Train on 143614 samples, validate on 15957 samples
Epoch 1/15
143614/143614 [==============================] - 332s 2ms/step - loss: 0.0582 - acc: 0.9797 - val_loss: 0.0428 - val_acc: 0.9828

Epoch 00001: val_loss improved from inf to 0.04283, saving model to attngru_fold0.h5
Epoch 2/15
143614/143614 [==============================] - 327s 2ms/step - loss: 0.0437 - acc: 0.9831 - val_loss: 0.0404 - val_acc: 0.9839

Epoch 00002: val_loss improved from 0.04283 to 0.04038, saving model to attngru_fold0.h5
Epoch 3/15
143614/143614 [==============================] - 329s 2ms/step - loss: 0.0414 - acc: 0.9837 - val_loss: 0.0392 - val_acc: 0.9845

Epoch 00003: val_loss improved from 0.04038 to 0.03923, saving model to attngru_fold0.h5
Epoch 4/15
143614/143614 [==============================] - 329s 2ms/step - loss: 0.0398 - acc: 0.9843 - val_loss: 0.0393 - val_acc: 0.9844

Epoch 00004: val_loss did not improve
Epoch 5/15
143614/143614 [=======================

In [ ]:
CLASSES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print(len(test_predicts_list))
test_predicts_am = np.zeros(test_predicts_list[0].shape)

for fold_predict in test_predicts_list:
    test_predicts_am += fold_predict

test_predicts_am = (test_predicts_am / len(test_predicts_list))

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts_am = pd.DataFrame(data=test_predicts_am, columns=CLASSES)
test_predicts_am["id"] = test_ids
test_predicts_am = test_predicts_am[["id"] + CLASSES]
test_predicts_am.to_csv("10fold_deepmoji_am.csv", index=False)